In [1]:
from flask import Flask, request, jsonify
from keras.layers import (Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout)
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
import requests

Using TensorFlow backend.


In [2]:
class DQN:

    def __init__(self, interface):
        # For each sample we input the integer identifiers
        # of a single user and a single item
        user_id_input = Input(shape=(1,), name='user')
        item_id_input = Input(shape=(1,), name='item')
        metadata_input = Input(shape=(interface.nb_variables,), name='metadata')

        embedding_size = 30
        user_embedding = Embedding(output_dim=embedding_size, input_dim=interface.nb_users + 1,
                                   input_length=1, name='user_embedding')(user_id_input)

        item_embedding = Embedding(output_dim=embedding_size, input_dim=interface.nb_items + 1,
                                   input_length=1, name='item_embedding')(item_id_input)

        # reshape from shape: (batch_size, input_length, embedding_size)
        # to shape: (batch_size, input_length * embedding_size) which is
        # equal to shape: (batch_size, embedding_size)
        user_vecs = Flatten()(user_embedding)
        item_vecs = Flatten()(item_embedding)
        # metadata_vecs = Flatten()(metadata_input)

        conc = Concatenate(axis=1)([user_vecs, item_vecs, metadata_input])

        dense_1 = Dense(256, activation='relu')(conc)

        dropout_1 = Dropout(0.5)(dense_1)

        dense_2 = Dense(128, activation='relu')(dropout_1)

        dropout_2 = Dropout(0.5)(dense_2)

        y = Dense(1)(dropout_2)

        self.model = Model(inputs=[user_id_input, item_id_input,
            metadata_input], outputs=y)
        self.model.compile(optimizer='adam', loss='mean_squared_error')
        self.model.save('env2_initial_weight.h5')

    def reset(self):
        self.model = load_model('env2_initial_weight.h5')

    def train(self, user_id_train, item_id_train, metadata, rating_train):
        early_stopping = EarlyStopping(monitor='val_loss', patience=2)
        self.model.fit([user_id_train, item_id_train, metadata], rating_train,
                       batch_size=64, epochs=20, validation_split=0.1,
                       shuffle=True, callbacks=[early_stopping])

    def predict(self, user_id, item_id, metadata):
        return self.model.predict([user_id, item_id, metadata])

In [3]:
class Interface:

    def __init__(self, args):
        self.base_url = 'http://{}'.format(args.ip_address_env_1)
        self.user_id = args.user_id
        self.url_reset = '{}/reset'.format(self.base_url)
        self.url_predict = '{}/predict'.format(self.base_url)

        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()

        self.item_history = data['item_history']
        self.rating_history = data['rating_history']
        self.user_history = data['user_history']
        self.variables_history = data['variables_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']
        self.nb_variables = len(data['variables_history'][0])

        self.next_user = data['next_user']
        self.next_item = data['next_item']
        self.next_variables = data['next_variables']

    def reset(self):

        r = requests.get(url=self.url_reset, params={'user_id': self.user_id})
        data = r.json()

        self.item_history = data['item_history']
        self.rating_history = data['rating_history']
        self.user_history = data['user_history']
        self.variables_history = data['variables_history']

        self.nb_items = data['nb_items']
        self.nb_users = data['nb_users']
        self.nb_variables = len(data['variables_history'][0])

        self.next_user = data['next_user']
        self.next_item = data['next_item']
        self.next_variables = data['next_variables']

    def request(self, predicted_score):
        params = {}
        params['user_id'] = self.user_id
        params['predicted_score'] = predicted_score

        r = requests.get(url=self.url_predict, params=params)
        result = r.json()

        next_user = result['next_user']
        next_item = result['next_item']
        next_variables = result['next_variables']
        rating = result['rating']

        return next_user, next_item, next_variables, rating

In [4]:
class Trainer:

    def __init__(self, interface):
        self.interface = interface
        self.dqn = DQN(interface)

    def train(self):
        item_history = self.interface.item_history
        rating_history = self.interface.rating_history
        user_history = self.interface.user_history
        variables_history = self.interface.variables_history
        self.dqn.train(user_history, item_history, variables_history, rating_history)

    def reset(self):
        self.dqn.reset()
        self.train()

    def predict(self, user_id, item_id, metadata):
        return self.dqn.predict(user_id, item_id, metadata).item()

In [5]:
class Argument:
    pass

args = Argument
args.user_id = 'R3EIFXNYY6XMBXBR01BK'
args.ip_address_env_0 = '52.47.62.31'
args.ip_address_env_1 = '35.180.254.42'
args.ip_address_env_2 = '35.180.178.243'

interface = Interface(args)
trainer = Trainer(interface)
interface.reset()
trainer.reset()

Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 1s 296us/step - loss: 5.2458 - val_loss: 2.6063
Epoch 2/20
1800/1800 [==============================] - 0s 76us/step - loss: 2.7366 - val_loss: 2.2548
Epoch 3/20
1800/1800 [==============================] - 0s 72us/step - loss: 2.4816 - val_loss: 1.9325
Epoch 4/20
1800/1800 [==============================] - 0s 75us/step - loss: 1.9888 - val_loss: 1.5012
Epoch 5/20
1800/1800 [==============================] - 0s 75us/step - loss: 1.4623 - val_loss: 1.1782
Epoch 6/20
1800/1800 [==============================] - 0s 81us/step - loss: 1.2174 - val_loss: 1.0559
Epoch 7/20
1800/1800 [==============================] - 0s 78us/step - loss: 1.0940 - val_loss: 1.0241
Epoch 8/20
1800/1800 [==============================] - 0s 77us/step - loss: 0.9929 - val_loss: 1.0213
Epoch 9/20
1800/1800 [==============================] - 0s 78us/step - loss: 0.9647 - val_loss: 0.9857
Epoch 10/20
1800/1800 [==